In [1]:
RunningInCOLAB = 'google.colab' in str(get_ipython())
if RunningInCOLAB:
    !git clone https://github.com/MJC598/Neuron_Burst_Analysis.git

In [186]:
%matplotlib widget
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import scipy.io
import random
import time
import pandas as pds
from sklearn.metrics import r2_score

torch.manual_seed(0)

![Fully Connected Network](graphs/fullyConnectedNetwork.png)

In [187]:
class FCN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FCN,self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.tanh = nn.Tanh()
#         self.fc2 = nn.Linear(hidden_size, 40)
#         self.fc3 = nn.Linear(40, 15)
        self.fc2 = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        x = self.tanh(self.fc1(x))
#         x = self.tanh(self.fc2(x))
#         x = self.tanh(self.fc3(x))
        out = self.fc2(x)
        return out

In [188]:
class Conv1dFCN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Conv1dFCN, self).__init__()
        self.cn1 = nn.Conv1d(1, 1, kernel_size=5,padding=2)
        self.cn2 = nn.Conv1d(1, 1, kernel_size=3,padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(2)
        self.fc1 = nn.Linear(int(input_size/4), hidden_size)
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self,x):
        x = self.pool(self.relu(self.cn1(x)))
        x = self.pool(self.relu(self.cn2(x)))
#         print(x.size())
        x = self.tanh(self.fc1(x))
        out = self.fc2(x)
        return out

In [203]:
FRONT_TIME = -50
BACK_TIME = 40
T_START = 50+FRONT_TIME
T_END = 'Variable'
MODEL = FCN
OUTPUT = 'FR_LFP'
LOSS_FILE = ('losses/bursts/losses_' + str(MODEL) + 
             '_' + OUTPUT + '_' + str(FRONT_TIME) + 
             '_' + str(T_END) + '_30ms_avgaff_and_fr.csv')
PATH = ('models/bursts/' + str(MODEL) + '_' + OUTPUT + 
        '_' + str(FRONT_TIME) + '_' + str(T_END) + 
        '_30ms_avgaff_and_fr.pth')
# PATH = 'models/<class \'__main__.FCN\'>_FR_LFP_-50_90_full.pth'

DATA_PATH = 'data/bursts/burst_separatePNITNv3.mat'
COLAB_PRE = 'Neuron_Burst_Analysis/'
if RunningInCOLAB:
    LOSS_FILE = COLAB_PRE + LOSS_FILE
    PATH = COLAB_PRE + PATH
    DATA_PATH = COLAB_PRE + DATA_PATH

# Specific Model Parameters
input_features = 1
previous_time = 10
input_size = input_features * previous_time
hidden_size = 95
output_size = 1
batch_size = 1024
num_layers = 1
batch_first = True
dropout = 0.0
epochs = 50

### Build Data From Matlab File

Read in Matlab info and Average over sliding window of size 3 PN Afferent and PN Firing Rate. 
Variables:
$$
x = \text{PN Firing Rate}\\
y = \text{ITN Firing Rate}\\
z = \text{Local Field Potential}\\
e_{1} = \text{PN Afferent}\\
e_{2} = \text{ITN Afferent}\\
e_{3} = \text{PN Excitatory Point Conductance}\\
e_{4} = \text{ITN Excitatory Point Conductance}\\
e_{5} = \text{PN Inhibitory Point Conductance}\\
e_{6} = \text{ITN Inhibitory Point Conductance}\\
t = \text{Timestep}\\
m = \text{Number of Previous Timesteps}\\
N = \text{Number of Samples}\\
n = \text{Individual Sample}\\
f = \text{Number of Features}\\
\omega(n) = \text{Time Sequence of Sample n}\\
$$

Sliding Window:
$$
x_{t} = 
\begin{cases} 
    \frac{1}{3}\sum_{i=0}^{2}x_{t+i} & \text{if } t \geq 2\\
    x_{t} & otherwise\\
\end{cases}\\
e_{1,t} = 
\begin{cases}
    \frac{1}{3}\sum_{i=0}^{2}e_{1,t+i}& \text{if } t \geq 2\\
    e_{1,t} & otherwise\\
\end{cases}\\
$$

Data $\text{size} = (N \times 1) , n = (\omega(n) \times f)$:
$$
\begin{bmatrix}
x_{t} & y_{t} & e_{1, t} & e_{2, t} & e_{3, t} & e_{4, t} & e_{5, t} & e_{6, t} & z_{t}\\
x_{t+1} & y_{t+1} & e_{1, t+1} & e_{2, t+1} & e_{3, t+1} & e_{4, t+1} & e_{5, t+1} & e_{6, t+1} & z_{t+1}\\
...\\
x_{t+\omega(n)} & y_{t+\omega(n)} & e_{1, t+\omega(n)} & e_{2, t+\omega(n)} & e_{3, t+\omega(n)} & e_{4, t+\omega(n)} & e_{5, t+\omega(n)} & e_{6, t+\omega(n)} & z_{t+\omega(n)}
\end{bmatrix}
$$

Label $\text{size} = (N \times 1) , n = (\omega(n) \times 3)$:
$$
\begin{bmatrix}
x_{t} & y_{t} & z_{t}\\
x_{t+1} & y_{t+1} & z_{t+1}\\
...\\
x_{t+\omega(n)} & y_{t+\omega(n)} & z_{t+\omega(n)}
\end{bmatrix}
$$

Building Inputs $\text{size} = ((\sum_{n=0}^{N}\omega(n) - m - 1) \times (f*m))$:
$$
\begin{bmatrix}
x_{0,t} & y_{0,t} & e_{1,0,t} & e_{2,0,t} & e_{3,0,t} & e_{4,0,t} & e_{5,0,t} & e_{6,0,t} & z_{0,t} & x_{0,t+1} & ... & z_{0,t+m}\\
x_{0,t+1} & y_{0,t+1} & e_{1,0,t+1} & e_{2,0,t+1} & e_{3,0,t+1} & e_{4,0,t+1} & e_{5,0,t+1} & e_{6,0,t+1} & z_{0,t+1} & x_{0,t+2} & ... & z_{0,t+m+1}\\
...\\
x_{0,\omega(0)-m-1} & y_{0,\omega(0)-m-1} & e_{1,0,\omega(0)-m-1} & e_{2,0,\omega(0)-m-1} & e_{3,0,\omega(0)-m-1} & e_{4,0,\omega(0)-m-1} & e_{5,0,\omega(0)-m-1} & e_{6,0,\omega(0)-m-1} & z_{0,\omega(0)-m-1} & x_{0,\omega(0)-m} & ... & z_{0,\omega(0)-1}\\
...\\
x_{N,t} & y_{N,t} & e_{1,N,t} & e_{2,N,t} & e_{3,N,t} & e_{4,N,t} & e_{5,N,t} & e_{6,N,t} & z_{N,t} & x_{N,t+1} & ... & z_{N,t+m}\\
x_{N,t+1} & y_{N,t+1} & e_{1,N,t+1} & e_{2,N,t+1} & e_{3,N,t+1} & e_{4,N,t+1} & e_{5,N,t+1} & e_{6,N,t+1} & z_{N,t+1} & x_{N,t+2} & ... & z_{N,t+m+1}\\
...\\
x_{N,\omega(N)-m-1} & y_{N,\omega(N)-m-1} & e_{1,N,\omega(N)-m-1} & e_{2,N,\omega(N)-m-1} & e_{3,N,\omega(N)-m-1} & e_{4,N,\omega(N)-m-1} & e_{5,N,\omega(N)-m-1} & e_{6,N,\omega(N)-m-1} & z_{N,\omega(N)-m-1} & x_{N,\omega(N)-m} & ... & z_{N,\omega(N)-1}\\
\end{bmatrix}
$$

Building Labels $\text{size} = ((\sum_{n=0}^{N}\omega(n) - m - 1) \times 3)$:
$$
\begin{bmatrix}
x_{0,t+m+1} & y_{0,t+m+1} & z_{0,t+m+1}\\
x_{0,t+m+2} & y_{0,t+m+2} & z_{0,t+m+2}\\
...\\
x_{0,\omega(0)} & y_{0,\omega(0)} & z_{0,\omega(0)}\\
...\\
x_{N,t+m+1} & y_{N,t+m+1} & z_{N,t+m+1}\\
x_{N,t+m+2} & y_{N,t+m+2} & z_{N,t+m+2}\\
...\\
x_{N,\omega(N)} & y_{N,\omega(N)} & z_{N,\omega(N)}\\
\end{bmatrix}
$$

In [204]:
from scipy.stats import boxcox
def clean_data(data):
    data = np.transpose(data)
    data += 1.0 + (-1.0*np.min(data))
    data, _ = boxcox(np.squeeze(data))
    return np.diff(data, n=2, axis=0, prepend=[0], append=[0])

In [205]:
def get_full_data_from_mat(file_path, output_index=None, type='pre_pn'):
    data = scipy.io.loadmat(file_path)
    data2 = scipy.io.loadmat('data/bursts/nonburst_separatePNITNv1.mat')

    full_labels = []
    full_data = []
    
    for i in range(1, data['info_collect'].shape[0]):
        arr = data['info_collect'][i]
        
        pnfr = np.row_stack((arr[2], arr[11]))
        pnaff = np.row_stack((arr[4], arr[13]))
        
        for j in range(pnfr.shape[0]):
            if j >= 2:
                pnfr[j,:] = (pnfr[j-2,:] + pnfr[j-1,:] + pnfr[j,:])/3
                pnaff[j,:] = (pnaff[j-2,:] + pnaff[j-1,:] + pnaff[j,:])/3
            else:
                pnfr[j,:] = pnfr[j,:]
                pnaff[j,:] = pnaff[j,:]
        
#         full_labels.append(np.column_stack((pnfr, 
#                                             np.row_stack((arr[3], arr[12])).astype('float'),
#                                             np.row_stack((arr[10], arr[19]))
#                                            ))
#                           )
        
#         full_data.append(np.column_stack((pnfr, 
#                                           np.row_stack((arr[3], arr[12])), 
#                                           pnaff, 
#                                           np.row_stack((arr[5], arr[14])), 
#                                           np.row_stack((arr[6], arr[15])), 
#                                           np.row_stack((arr[7], arr[16])), 
#                                           np.row_stack((arr[8], arr[17])), 
#                                           np.row_stack((arr[9], arr[18])),
#                                           np.row_stack((arr[10], arr[19]))
#                                          ))
#                         )

        full_labels.append(#np.column_stack((#pnfr, 
                                            #np.row_stack((arr[3], arr[12])).astype('float'),
                                            np.row_stack((arr[10], arr[19]))
                                          # ))
                          )
        
        full_data.append(np.column_stack((#pnfr, 
                                          #np.row_stack((arr[3], arr[12])), 
                                          pnaff, 
                                          np.row_stack((arr[5], arr[14])),
                                          np.row_stack((arr[10], arr[19]))
                                         ))
                        )
#     print(data2)
    for i in range(1, data2['info_collect_insig'].shape[0]):
        arr = data2['info_collect_insig'][i]
        
        pnfr = np.row_stack((arr[2], arr[11]))
        pnaff = np.row_stack((arr[4], arr[13]))
        
        for j in range(pnfr.shape[0]):
            if j >= 2:
                pnfr[j,:] = (pnfr[j-2,:] + pnfr[j-1,:] + pnfr[j,:])/3
                pnaff[j,:] = (pnaff[j-2,:] + pnaff[j-1,:] + pnaff[j,:])/3
            else:
                pnfr[j,:] = pnfr[j,:]
                pnaff[j,:] = pnaff[j,:]

        full_labels.append(np.row_stack((arr[10], arr[19])))
        
        full_data.append(np.column_stack((pnaff, 
                                          np.row_stack((arr[5], arr[14])),
                                          np.row_stack((arr[10], arr[19]))
                                         ))
                        )

    full_data = np.asarray(full_data,dtype=object)
    full_labels = np.asarray(full_labels,dtype=object)
    
    random.seed(10)
#     print(data2['info_collect_insig'].shape[0] + data['info_collect'].shape[0])
    data_samples = 10998 #5498 
    k = 8250
    
    ahead=output_size
    training_data = full_data[:k] 
    validation_data = full_data[k:data_samples]
    training_labels = full_labels[:k]
    validation_labels = full_labels[k:data_samples]
    
    td = [] 
    tl = []
    vd = []
    vl = []
    for i, d in enumerate(training_data):
        sample = d[:-1,:]
        label = training_labels[i][1:,:]
        for j in range(sample.shape[0]-(previous_time+ahead)):
            t = sample[j:j+previous_time,:]
            td.append(t.reshape((1,-1)))
            t2 = label[j+previous_time:j+previous_time+ahead,:]
            tl.append(t2.reshape((1,-1)))
    td = np.expand_dims(np.vstack(td), axis=1)
    print(td.shape)
    tl = np.expand_dims(np.vstack(tl), axis=1)
    
    
    for i, d in enumerate(validation_data):
        sample = d[:-1,:]
        label = validation_labels[i][1:,:]
        for j in range(sample.shape[0]-(previous_time+ahead)):
            t = sample[j:j+previous_time,:]
            vd.append(t.reshape((1,-1)))
            t2 = label[j+previous_time:j+previous_time+ahead,:]
            vl.append(t2.reshape((1,-1)))
    vd = np.expand_dims(np.vstack(vd), axis=1)
    vl = np.expand_dims(np.vstack(vl), axis=1)
    print(vd.shape)

    training_dataset = TensorDataset(torch.Tensor(td), torch.Tensor(tl))
    validation_dataset = TensorDataset(torch.Tensor(vd), torch.Tensor(vl))

    return training_dataset, validation_dataset
# get_full_data_from_mat(DATA_PATH)

In [206]:
from numpy.random import default_rng
def get_raw_data_from_mat():
    lfp_file = 'data/raw_data/LFP_filt.txt'
    fir_file = 'data/raw_data/FR_PN_ITN.txt'
    aff_file = 'data/raw_data/AFF_PN_ITN.txt'
    with open(lfp_file) as f:
        lfp = f.read().splitlines()
    lfp = np.array([float(x) for x in lfp]).reshape((-1, 1))
        
#     with open(fir_file) as f:
#         fr = f.read().splitlines()
#     fr = np.array([(float(x.split(',')[0]), float(x.split(',')[1])) for x in fr])
        
    with open(aff_file) as f:
        aff = f.read().splitlines()
    aff = np.array([(float(x.split(',')[0]), float(x.split(',')[1])) for x in aff])
        
#     full_data = np.hstack((lfp, aff))
    full_data = lfp
    full_labels = lfp
    
    rng = default_rng(seed=67)
    num_feat = input_features
    pred_time = 5
    training_samples = 300000
    indices = rng.integers(low=0, high=full_labels.shape[0]-(previous_time+pred_time), size=training_samples)
    validation_samples = 100000
    v_indices = rng.integers(low=0, high=full_labels.shape[0]-(previous_time+pred_time), size=validation_samples)
    training_data = []
    training_labels = []
    validation_data = []
    validation_labels = []
    f_data = []
    f_labels = []
    
    for idx in indices:
        training_data.append(full_data[idx:idx+previous_time,:].reshape((-1,previous_time*num_feat)))
        training_labels.append(full_labels[idx+previous_time+4,:].reshape((-1,output_size)))
    training_data = np.stack(training_data, axis=0)
    training_labels = np.stack(training_labels, axis=0)
    
    for idx in v_indices:
        validation_data.append(full_data[idx:idx+previous_time,:].reshape((-1,previous_time*num_feat)))
        validation_labels.append(full_labels[idx+previous_time+4,:].reshape((-1,output_size)))
    validation_data = np.stack(validation_data, axis=0)
    validation_labels = np.stack(validation_labels, axis=0)
    
    for i in range(full_data.shape[0]-(previous_time+pred_time)):
        f_data.append(full_data[i:i+previous_time,:].reshape((-1,previous_time*num_feat)))
        f_labels.append(full_labels[i+previous_time+4,:].reshape((-1,output_size)))
    f_data = np.stack(f_data, axis=0)
    f_labels = np.stack(f_labels, axis=0)
    
    print('Training Data: {}'.format(training_data.shape))
    print('Training Labels: {}'.format(training_labels.shape))
    print('Validation Data: {}'.format(validation_data.shape))
    print('Validation Labels: {}'.format(validation_labels.shape))
    print('Full Data: {}'.format(f_data.shape))
    print('Full Labels: {}'.format(f_labels.shape))
    
    training_dataset = TensorDataset(torch.Tensor(training_data), torch.Tensor(training_labels))
    validation_dataset = TensorDataset(torch.Tensor(validation_data), torch.Tensor(validation_labels))
    f_dataset = TensorDataset(torch.Tensor(f_data), torch.Tensor(f_labels))

    return training_dataset, validation_dataset, f_dataset
# get_raw_data_from_mat()

In [207]:
def train_model(model,save_filepath,training_loader,validation_loader,epochs,device):
    epochs_list = []
    train_loss_list = []
    val_loss_list = []
    training_len = len(training_loader.dataset)
    validation_len = len(validation_loader.dataset)
    
#     feedback_arr = torch.zeros(batch_size, 90)
    
    #splitting the dataloaders to generalize code
    data_loaders = {"train": training_loader, "val": validation_loader}
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_func = nn.MSELoss()
#     loss_func = nn.L1Loss()
    decay_rate = 0.90 #decay the lr each step to 93% of previous lr
    lr_sch = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decay_rate)

    total_start = time.time()

    """
    You can easily adjust the number of epochs trained here by changing the number in the range
    """
    for epoch in tqdm(range(epochs), position=0, leave=True):
        start = time.time()
        train_loss = 0.0
        val_loss = 0.0
        temp_loss = 100000000000000.0
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            for i, (x, y) in enumerate(data_loaders[phase]):
                x = x.to(device)
                output = model(x)
                y = y.to(device)
                loss = loss_func(torch.squeeze(output), torch.squeeze(y)) 
                #backprop             
                optimizer.zero_grad()           
                if phase == 'train':
                    loss.backward()
                    optimizer.step()                                      

                #calculating total loss
                running_loss += loss.item()
            
            if phase == 'train':
                train_loss = running_loss
                lr_sch.step()
            else:
                val_loss = running_loss

        end = time.time()
        # shows total loss
        if epoch%5 == 0:
            print('[%d, %5d] train loss: %.6f val loss: %.6f' % (epoch + 1, i + 1, train_loss, val_loss))
#         print(end - start)
        
        #saving best model
        if val_loss < temp_loss:
            torch.save(model, save_filepath)
            temp_loss = val_loss
        epochs_list.append(epoch)
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
    total_end = time.time()
#     print(total_end - total_start)
    #Creating loss csv
    loss_df = pds.DataFrame(
        {
            'epoch': epochs_list,
            'training loss': train_loss_list,
            'validation loss': val_loss_list
        }
    )
    # Writing loss csv, change path to whatever you want to name it
    
    loss_df.to_csv(LOSS_FILE, index=None)
    return train_loss_list, val_loss_list

In [208]:
# f_tr, f_va = get_full_data_from_mat(DATA_PATH)
f_tr, f_va, f_data = get_raw_data_from_mat()

# Turn datasets into iterable dataloaders
f_training_loader = DataLoader(dataset=f_tr,batch_size=batch_size)
f_validation_loader = DataLoader(dataset=f_va,batch_size=batch_size)
f_dataset_loader = DataLoader(dataset=f_data,batch_size=batch_size)

Training Data: (300000, 1, 10)
Training Labels: (300000, 1, 1)
Validation Data: (100000, 1, 10)
Validation Labels: (100000, 1, 1)
Full Data: (2999986, 1, 10)
Full Labels: (2999986, 1, 1)


In [209]:
model1 = MODEL(input_size,hidden_size,output_size)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model1.to(device)

pnfr_training_loss, pnfr_validation_loss = train_model(model1,PATH,f_training_loader,
                                                       f_validation_loader,epochs,device)

  0%|          | 0/50 [00:00<?, ?it/s]

[1,    98] train loss: 0.094020 val loss: 0.003100
[6,    98] train loss: 0.008069 val loss: 0.002571
[11,    98] train loss: 0.006749 val loss: 0.002103
[16,    98] train loss: 0.005413 val loss: 0.001743
[21,    98] train loss: 0.004867 val loss: 0.001566
[26,    98] train loss: 0.004643 val loss: 0.001591
[31,    98] train loss: 0.004542 val loss: 0.001508
[36,    98] train loss: 0.004493 val loss: 0.001492
[41,    98] train loss: 0.004453 val loss: 0.001492
[46,    98] train loss: 0.004438 val loss: 0.001475


In [210]:
def r2_full_score_eval(model, testing_dataloader, k=None):
    output_list = []
    labels_list = []
    temp_list = []
    for i, (x, y) in enumerate(testing_dataloader):
        output = model(x)         
        output_list.append(output.detach().cpu().numpy())
        labels_list.append(y.detach().cpu().numpy())
        if k != None and i == k-1:
            break
#     print("Output list size: {}".format(len(output_list)))
#     print(output_list[0].shape)
    output_list = np.squeeze(np.concatenate(output_list, axis=0))
#     print(output_list.shape)
    labels_list = np.squeeze(np.concatenate(labels_list, axis=0))
#     print(labels_list.shape)
#     print(output_list.shape)
#     print(labels_list.shape)
    print(r2_score(labels_list, output_list))
    return output_list, labels_list

### Mask
Variables:
$$
x = \text{PN Firing Rate}\\
y = \text{ITN Firing Rate}\\
z = \text{Local Field Potential}\\
t = \text{Timestep}\\
m = \text{Number of Previous Timesteps}\\
e_{x} = \text{External Input}\\
$$


Output Array:
$$
\begin{bmatrix}
x_{n,t+m} & y_{n,t+m} & z_{n,t+m}\\
\end{bmatrix}
$$
Feedback Array: 
$$
\begin{bmatrix}
x_{n,t-1} & y_{n,t-1} & z_{n,t-1} & x_{n,t} & ... & z_{n,t+m-1}\\
\end{bmatrix}
$$
Replace index 0, 1, 2 with Output Array and Roll

Rolled Feedback Array: 
$$
\begin{bmatrix}
x_{n,t} & y_{n,t} & z_{n,t} & x_{n,t+1} & ... & z_{n,t+m}\\
\end{bmatrix}
$$
Input Array:
$$
\begin{bmatrix}
x_{n,t} & y_{n,t} & e_{1,n,t} & e_{2,n,t} & e_{3,n,t} & e_{4,n,t} & e_{5,n,t} & e_{6,n,t} & z_{n,t} & x_{n,t+1} & ... & z_{n,t+m}\\
\end{bmatrix}
$$
New Output:
$$
\begin{bmatrix}
x_{n,t+m+1} & y_{n,t+m+1} & z_{n,t+m+1}\\
\end{bmatrix}
$$

In [211]:
def r2_score_eval(model, testing_dataloader, start=10, k=None):
    output_list = []
    labels_list = []
    temp_list = []
    mask = [False, False, True]
    mask = list(np.tile(mask, previous_time))
    feedback_arr = torch.zeros(1, 1*previous_time)
    for i, (x, y) in enumerate(testing_dataloader):
        x2 = x.numpy().copy()
        if i >= start:
            x2[:,0,mask] = feedback_arr.numpy()
        output = model(torch.Tensor(x2))     
        
        feedback_arr = torch.roll(feedback_arr, -1, 1)
        feedback_arr[:,0] = output[:,0,0].detach()
#         feedback_arr[:,1] = output[:,0,1].detach()
#         feedback_arr[:,2] = output[:,0,2].detach()
        feedback_arr.detach()
        
        output_list.append(output.detach().cpu().numpy())
        labels_list.append(y.detach().cpu().numpy())
        if k != None and i == k-1:
            break
#     print("Output list size: {}".format(len(output_list)))
#     print(output_list[0].shape)
    output_list = np.squeeze(np.concatenate(output_list, axis=0))
    labels_list = np.squeeze(np.concatenate(labels_list, axis=0))
    print(r2_score(labels_list, output_list))
    return output_list, labels_list

In [212]:
model1 = torch.load(PATH)
model1.eval()

start = 10
k = 10000
end= (start + k) if k != None else None

model1.to('cpu')

ft_output_list, ft_labels_list = r2_full_score_eval(model1, f_training_loader,end)
fv_output_list, fv_labels_list = r2_full_score_eval(model1, f_validation_loader,end)
f_output_list, f_labels_list = r2_full_score_eval(model1, f_dataset_loader, end)

0.9833163188294743
0.9835364546702573
0.9834315104692611


In [217]:
fig1, ax1 = plt.subplots(nrows=1, ncols=2)
fig1.tight_layout()
ax1[0].plot(range(epochs), pnfr_training_loss)
ax1[0].set_title('Validation Loss')
ax1[0].set_ylabel('Loss')
ax1[0].set_xlabel('Epoch')

ax1[1].plot(range(epochs), pnfr_validation_loss)
ax1[1].set_title('Training Loss')
ax1[1].set_ylabel('Loss')
ax1[1].set_xlabel('Epoch')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 23.52222222222222, 'Epoch')

In [218]:
fig, ax = plt.subplots(nrows=1, ncols=2)
fig.tight_layout()

ax[0].plot(np.arange(start-output_size,end), fv_labels_list[start-output_size:end], color='blue',label='Labels')
# ax[2,0].plot(np.arange(start-10,end), v_output_list[start-10:end,2], color='red',label='Internal Loop')
ax[0].scatter(np.arange(start-output_size,end), fv_output_list[start-1:end], color='slateblue',label='Training t+1')
# ax[0].scatter(np.arange(start-output_size,end), fv_output_list[start-2:end+8,1], color='lightsteelblue',label='Training t+2')
# ax[0].scatter(np.arange(start-output_size,end), fv_output_list[start-3:end+7,2], color='gray',label='Training t+3')
# ax[0].scatter(np.arange(start-output_size,end), fv_output_list[start-4:end+6,3], color='sienna',label='Training t+4')
# ax[0].scatter(np.arange(start-output_size,end), fv_output_list[start-5:end+5,4], color='magenta',label='Training t+5')
# ax[0].scatter(np.arange(start-10,end), fv_output_list[start-6:end+4,5], color='aquamarine',label='Training t+6')
# ax[0].scatter(np.arange(start-10,end), fv_output_list[start-7:end+3,6], color='darkorange',label='Training t+7')
# ax[0].scatter(np.arange(start-10,end), fv_output_list[start-8:end+2,7], color='brown',label='Training t+8')
# ax[0].scatter(np.arange(start-10,end), fv_output_list[start-9:end+1,8], color='purple',label='Training t+9')
# ax[0].plot(np.arange(start-10,end), fv_output_list[start-10:end], color='green',label='Training t+10')


ax[0].set_title('Validation LFP')
ax[0].set_ylabel('LFP')
ax[0].set_xlabel('Time')
# ax[2,0].legend()

ax[1].plot(np.arange(start-output_size,end), ft_labels_list[start-output_size:end], color='blue',label='Labels')
# a[2,1].plot(np.arange(start-10,end), t_output_list[start-10:end,2], color='red',label='Internal Loop')
ax[1].scatter(np.arange(start-output_size,end), ft_output_list[start-1:end], color='slateblue',label='Training t+1')
# ax[1].scatter(np.arange(start-output_size,end), ft_output_list[start-2:end+8,1], color='lightsteelblue',label='Training t+2')
# ax[1].scatter(np.arange(start-output_size,end), ft_output_list[start-3:end+7,2], color='gray',label='Training t+3')
# ax[1].scatter(np.arange(start-output_size,end), ft_output_list[start-4:end+6,3], color='sienna',label='Training t+4')
# ax[1].scatter(np.arange(start-output_size,end), ft_output_list[start-5:end+5,4], color='magenta',label='Training t+5')
# ax[1].scatter(np.arange(start-10,end), ft_output_list[start-6:end+4,5], color='aquamarine',label='Training t+6')
# ax[1].scatter(np.arange(start-10,end), ft_output_list[start-7:end+3,6], color='darkorange',label='Training t+7')
# ax[1].scatter(np.arange(start-10,end), ft_output_list[start-8:end+2,7], color='brown',label='Training t+8')
# ax[1].scatter(np.arange(start-10,end), ft_output_list[start-9:end+1,8], color='purple',label='Training t+9')
# ax[1].plot(np.arange(start-10,end), ft_output_list[start-10:end], color='green',label='Training t+10')

ax[1].set_title('Training LFP')
ax[1].set_ylabel('LFP')
ax[1].set_xlabel('Time')
ax[1].legend()

# import plotly.tools as tls
# plotly_fig = tls.mpl_to_plotly(fig)
# plotly_fig.write_html("testfile.html")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [221]:
fig, ax = plt.subplots(nrows=1, ncols=1)
fig.tight_layout()

ax.plot(np.arange(start,end), f_labels_list[start:end], color='blue',label='Labels')
ax.scatter(np.arange(start,end), f_output_list[start:end], color='red',label='Training t+10')


ax.set_title('Validation LFP')
ax.set_ylabel('LFP')
ax.set_xlabel('Time')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [216]:
plt.close('all')